In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/10 02:40:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import functions as F

In [3]:
pyspark.__version__

'3.4.4'

In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-04-10 02:41:00--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.112, 3.164.82.197, 3.164.82.40, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet.2’

yellow_tripdata_202 100%[===================>]  61.36M  19.5MB/s    in 4.2s    

2025-04-10 02:41:05 (14.6 MB/s) - ‘yellow_tripdata_2024-10.parquet.2’ saved [64346071/64346071]



In [5]:
df = spark.read.parquet("yellow_tripdata_2024-10.parquet")

In [6]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [7]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [9]:
df.repartition(4).write.parquet("data/pq/yellow/2024/10", mode='overwrite')

In [10]:
!ls -lh data/pq/yellow/2024/10

total 90M
-rw-r--r-- 1 sweantu sweantu   0 Apr 10 02:41 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 23M Apr 10 02:41 part-00000-91028356-783e-4f5b-992a-9105b7ac5091-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 23M Apr 10 02:41 part-00001-91028356-783e-4f5b-992a-9105b7ac5091-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 23M Apr 10 02:41 part-00002-91028356-783e-4f5b-992a-9105b7ac5091-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 23M Apr 10 02:41 part-00003-91028356-783e-4f5b-992a-9105b7ac5091-c000.snappy.parquet


In [11]:
df.createOrReplaceTempView("yellow_tripdata")

In [12]:
spark.sql("""
SELECT
    COUNT(1)
FROM yellow_tripdata
WHERE
    tpep_pickup_datetime IS NOT NULL
    AND tpep_pickup_datetime >= '2024-10-15 00:00:00'
    AND tpep_pickup_datetime < '2024-10-16 00:00:00'
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [13]:
spark.sql(""" 
SELECT count(*)
FROM yellow_tripdata
WHERE cast(tpep_pickup_datetime as date) = '2024-10-15'
and tpep_pickup_datetime is not null
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [14]:
df.withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime)) \
    .filter("pickup_date = '2024-10-15'") \
    .count()

128893

In [15]:
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [16]:
df.withColumn('duration', 
    (F.unix_timestamp(df.tpep_dropoff_datetime) - F.unix_timestamp(df.tpep_pickup_datetime)) / 60 / 60
) \
.orderBy('duration', ascending=False) \
.limit(5) \
.select("duration") \
.show()

+------------------+
|          duration|
+------------------+
| 162.6177777777778|
|           143.325|
|137.76055555555556|
|114.83472222222221|
| 89.89833333333333|
+------------------+



In [17]:
spark.sql("""
SELECT 
    DATEDIFF(hour, tpep_dropoff_datetime, tpep_pickup_datetime) AS duration
FROM yellow_tripdata
ORDER BY 1
LIMIT 5
""").show()

+--------+
|duration|
+--------+
|    -162|
|    -143|
|    -137|
|    -114|
|     -89|
+--------+



In [18]:
df_zone = spark.read.option("header", "true").csv("taxi_zone_lookup.csv")

In [19]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [20]:
df_zone.createOrReplaceTempView('zone')

In [21]:
spark.sql("""
SELECT
    z.Zone,
    COUNT(1) as count
FROM
    yellow_tripdata y
JOIN zone z ON y.PULocationID = z.LocationID
GROUP BY
     z.Zone
ORDER BY
    count
""").take(5)

[Row(Zone="Governor's Island/Ellis Island/Liberty Island", count=1),
 Row(Zone='Rikers Island', count=2),
 Row(Zone='Arden Heights', count=2),
 Row(Zone='Jamaica Bay', count=3),
 Row(Zone='Green-Wood Cemetery', count=3)]

In [22]:
(df.alias('y')
 .join(df_zone.alias('z'), df['PULocationID'] == df_zone['LocationID'])
 .groupBy('z.Zone')
 .count()
 .orderBy('count')
 .take(5))

[Row(Zone="Governor's Island/Ellis Island/Liberty Island", count=1),
 Row(Zone='Rikers Island', count=2),
 Row(Zone='Arden Heights', count=2),
 Row(Zone='Jamaica Bay', count=3),
 Row(Zone='Green-Wood Cemetery', count=3)]

In [23]:
df_result = df.join(df_zone, df.PULocationID == df_zone.LocationID).groupBy('Zone').count().withColumnRenamed('count', 'cnt').orderBy('cnt').limit(5)

In [24]:
df_result.show()

+--------------------+---+
|                Zone|cnt|
+--------------------+---+
|Governor's Island...|  1|
|       Rikers Island|  2|
|       Arden Heights|  2|
|         Jamaica Bay|  3|
| Green-Wood Cemetery|  3|
+--------------------+---+

